In [6]:
#import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# load the data
data = torch.load('data/brain_cancer_dataset_small.pt')

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [5]:
# split the data into training , validation and test sets
# in total we have 300 samples:
# 200 training samples
# 50 validation samples
# 50 test samples

# they are ordered, hence before splitting we shuffle the data
torch.manual_seed(0)
data = data[torch.randperm(len(data))]

# split the data
train_data = data[:200]
val_data = data[200:250]
test_data = data[250:]

In [7]:
# define the model

# architecture: 

# input -> conv1 -> maxpool -> conv2 -> maxpool -> conv3 -> maxpool -> flatten -> -> dense(sofmax)

class brain_classifier(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # Define layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(in_features=32*2*2, out_features=4)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        
        x = self.flatten(x)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        
        return x


In [ ]:
# Define the training loop
def train(model, train_data, val_data, epochs=10, lr=0.001):
    # Define the loss function
    criterion = nn.CrossEntropyLoss()
    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Store the losses
    train_losses = []
    val_losses = []
    
    # Move model to the appropriate device
    model.to(device)
    
    for epoch in range(epochs):
        # Set the model to training mode
        model.train()
        
        # Iterate over the training data
        for i, (x, y) in enumerate(train_data):
            # Move data to the appropriate device
            x, y = x.to(device), y.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            y_pred = model(x)
            # Compute the loss
            loss = criterion(y_pred, y)
            # Backward pass
            loss.backward()
            # Update the weights
            optimizer.step()
            # Store the loss
            train_losses.append(loss.item())
            # Print the loss
            if i % 10 == 0:
                print(f'Epoch {epoch}, Iteration {i}, Loss {loss.item()}')
        
        # Set the model to evaluation mode
        model.eval()
        # Compute the validation loss
        val_loss = 0
        with torch.no_grad():
            for x, y in val_data:
                # Move data to the appropriate device
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                val_loss += loss.item()
                
        val_losses.append(val_loss/len(val_data))
        
        print(f'Epoch {epoch}, Validation Loss {val_loss/len(val_data)}')
        
    return train_losses, val_losses

In [ ]:
# Example instantiation and training
model = brain_classifier()

In [ ]:
train_losses, val_losses = train(model, train_data, val_data, epochs=10, lr=0.001)

In [ ]:
# Plot the losses